In [3]:
%load_ext autoreload 
%autoreload 2

In [2]:
from preprocessing import ner_training, expansion, training_data_extraction
from postprocessing import trainingdata_generation, extract_new_entities, filtering
from config import ROOTPATH
from gensim.models import Doc2Vec
from elasticsearch import Elasticsearch

modeldoc2vec = Doc2Vec.load(ROOTPATH + '/models/doc2vec.model')
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

#  training_data_extraction.extract(seed)

In [85]:
seeds = [10]#, 10, 25, 50, 100]

"""
Extract training data for different number of seeds
"""
for seed in seeds:
    training_data_extraction.extract(seed)

Started training data extraction
['polycomb protein eed', 'von willebrand factor', 'coagulation factor ix', 'retinol-binding protein 3', 'hepatocyte nuclear factor 3-beta', 'signal transducer and activator of transcription 5b', 'double-strand break repair protein mre11a', 'ccaat/enhancer-binding protein gamma', 'bmp4', 'bmp receptor type-1a', 'myc protein', 'transcription factor sox-5 isoform 2', 'myoblast determination protein 1', 'apolipoprotein b-100', 'interferon gamma', 'caveolin-1', 'low affinity immunoglobulin gamma fc region receptor ii-b', 'tissue-type plasminogen activator', 'vascular endothelial growth factor receptor 2', 'ccaat/enhancer-binding protein delta', 'glucose-6-phosphate 1-dehydrogenase', 'a disintegrin and metalloproteinase with thrombospondin motifs 3', 'lethalmalignant brain tumor-like protein mbt protein, h-lmbt, l3mbtl1, lmbt protein homolog, lmbt-like', 'transcription factor e2f7', 'c-reactive protein', 'lactoylglutathione lyase', 'platelet glycoprotein 4', 

['PARP polyADP-ribose polymerase  , C Casp 3 cleaved caspase 3 b Lysates were harvested 2\xa0h after lapatinib treatment and were analyzed by immunoblot with the indicated antibodies.', 'The presence of these potential anticoagulant molecules may seem at odds with the proposed coagulation effects of some of the putative platypus venom serine proteases identified above  , but there are examples in snakes where one venom contains multiple proteases with coagulant and anticoagulant effects  , or where one protease has both effects; it is thought that in these cases the concentration of toxins determines the type of effect on the victim reviewed in 28. Many snake venoms also contain serine protease inhibitors  , which affect hemostasis and produce inflammation 40; toxin kunitz-type protease inhibitors called kalicludines are also found in sea anemones 41.', 'The function of protease inhibitors in platypus venom gland is unclear  , but it is suggested that perhaps these act to inhibit the c

['These results suggest that endogenous BCAR3 inhibits Smad signaling. In addition  , we did not observe remarkable effects of BCAR3 siRNA on p130Cas protein levels or TGFβ-induced p38 phosphorylation Figure\xa02b.', 'Our findings define a novel positive regulatory feedback loop through which TGFβ signaling further induces its effects by blocking expression of the Smad inhibitor BCAR3. We also found BCAR3 protein levels in breast cancer cells to be controlled by TGFβ  , as TGFβ treatment decreases BCAR3 expression in a Smad-dependent and proteasome-dependent manner.', 'For example  , EGL-4 likely functions upstream of the TGF-β pathway SMAD transcription factors DAF-3 and DAF-5  , as mutations in DAF-3 and DAF-5 suppress several egl-4 mutant phenotypes  , including chemosensory  , dauer formation  , egg laying and body size defects 47. Similar to PKGI and PKGII in mammalian systems  , whose targets include components of G protein-coupled signaling 39–42  , the c-Jun N-terminal kinases 

In [150]:
import random
from elasticsearch import Elasticsearch
import nltk
import re
from sklearn.model_selection import train_test_split
from nltk import tokenize
import config as cfg

numberOfSeeds = 10
    
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])    
print('Started training data extraction')

# First we get the dataset names which have been used in the testing set (TestB) to exclude them from the training sentences
X_testB = []
with open('data/protein-names-test.txt', 'r') as file:
    for row in file.readlines():
        X_testB.append(row.strip())
# lowercase the names
X_testB = [ds.lower() for ds in X_testB]
X_testB = list(set(X_testB))

Started training data extraction


In [153]:
# List of seed names
dsnames = []
corpuspath = "data/protein-names-train.txt"
with open(corpuspath, "r") as file:
    for row in file.readlines():
        dsnames.append(row.strip())
dsnames = [x.lower() for x in dsnames]
dsnames = list(set(dsnames))

In [161]:
# shuffle the list
X_train = random.sample(dsnames, numberOfSeeds)

paragraph = []

# using the seeds, extract the sentences from the publications using the query in elastic search
for dataset in X_train:
    datasetname = re.sub(r'\([^)]*\)', '', dataset)

    # Matching
    query = {"query":
        {"match": {
            "content.chapter.sentpositive": {
                "query": datasetname,
                "operator": "and"
            }
        }
        }
    }

    res = es.search(index="twosent", doc_type="twosentnorules",
                    body=query, size=10000)

    print(datasetname)
    print("Got %d Hits:" % res['hits']['total'])
    # clean up the sentences and if they dont contain the names of the testB then add them as the training data
    for doc in res['hits']['hits']:

        sentence = doc["_source"]["content.chapter.sentpositive"]

        words = nltk.word_tokenize(doc["_source"]["content.chapter.sentpositive"])
        lengths = [len(x) for x in words]
        average = sum(lengths) / len(lengths)

        if average < 3:
            continue

        sentence = sentence.replace("@ BULLET", "")
        sentence = sentence.replace("@BULLET", "")
        sentence = sentence.replace(", ", " , ")
        sentence = sentence.replace('(', '')
        sentence = sentence.replace(')', '')
        sentence = sentence.replace('[', '')
        sentence = sentence.replace(']', '')
        sentence = sentence.replace(',', ' ,')
        sentence = sentence.replace('?', ' ?')
        sentence = sentence.replace('..', '.')

        if any(ext in sentence.lower() for ext in X_testB):
            continue

        else:
            paragraph.append(sentence)

paragraph = list(set(paragraph))

n-cym protein
Got 0 Hits:
histidinol-phosphate aminotransferase
Got 0 Hits:
chordin
Got 203 Hits:
annexin a8
Got 0 Hits:
protein yipf4
Got 0 Hits:
nociceptin
Got 2 Hits:
malate dehydrogenase
Got 37 Hits:
d-galactonate dehydratase
Got 0 Hits:
tgf-beta 1
Got 20 Hits:
dipeptidase 1
Got 1 Hits:


In [162]:
# Split the data into training and testing and keep the sentences and also the seed names for annotation that will be used later

f1 = open(cfg.ROOTPATH + '/evaluation_files_prot/X_train_' + str(numberOfSeeds) + '_' + str(0) + '.txt', 'w')
for item in paragraph:
    f1.write(item)
f1.close()

f1 = open(cfg.ROOTPATH + '/evaluation_files_prot/X_Seeds_' + str(numberOfSeeds) + '_' + str(0) + '.txt', 'w')
for item in X_train:
    f1.write(item + '\n')
f1.close()

#  expansion.term_expansion_proteins

In [163]:
from numbers import Number
from sklearn.preprocessing import StandardScaler
import codecs, numpy
from sklearn.metrics import silhouette_score
import string
from preprocessing.Extract_NE import preprocess_NE
from sklearn.cluster import KMeans
from config import ROOTPATH
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet

numberOfSeeds, name, numberOfIteration, iteration = 10, 'term_expansion', str(0), str(0)

In [164]:
class autovivify_list(dict):
    '''Pickleable class to replicate the functionality of collections.defaultdict'''

    def __missing__(self, key):
        value = self[key] = []
        return value

    def __add__(self, x):
        '''Override addition for numeric types when self is empty'''
        if not self and isinstance(x, Number):
            return x
        raise ValueError

    def __sub__(self, x):
        '''Also provide subtraction method'''
        if not self and isinstance(x, Number):
            return -1 * x
        raise ValueError


def build_word_vector_matrix(vector_file, propernouns):
    '''Read a GloVe array from sys.argv[1] and return its vectors and labels as arrays'''
    numpy_arrays = []
    labels_array = []
    with codecs.open(vector_file, 'r', 'utf-8') as f:
        for c, r in enumerate(f):
            sr = r.split()
            try:
                if sr[0] in propernouns and not wordnet.synsets(sr[0]) and sr[0].lower() not in stopwords.words('english') and 'protein' not in sr[0].lower():
                    labels_array.append(sr[0])
                    numpy_arrays.append(numpy.array([float(i) for i in sr[1:]]))
            except:
                continue

    return numpy.array(numpy_arrays), labels_array

def find_word_clusters(labels_array, cluster_labels):
    '''Read the labels array and clusters label and return the set of words in each cluster'''
    cluster_to_words = autovivify_list()
    for c, i in enumerate(cluster_labels):
        cluster_to_words[i].append(labels_array[c])
    return cluster_to_words

In [93]:
"""
Term expansion approach for the first iteration
"""
# perform term expansion on the text of the training data using different number of seeds (i.e. 5,10,25,50,100)
for number in range(10):
    expansion.term_expansion_proteins(10, 'term_expansion', str(0), str(number))

started to extract general NE from text....
started to extract general NE from text....


KeyboardInterrupt: 

In [165]:
# in the first iteration use the initial text extracted using the seeds
if int(numberOfIteration) == 0:
    fileUnlabelled = ROOTPATH + '/evaluation_files_prot/X_train_' + str(numberOfSeeds) + '_' + str(iteration) + '.txt'

# in the next iterations use the text extracted using the new set of seeds
else:
    fileUnlabelled = ROOTPATH + '/evaluation_files_prot/' + name + 'text_Iteration' + numberOfIteration + str(
                     numberOfSeeds) + '_' + str(iteration) + '.txt'

'''Extract All the entities from the text'''
propernouns = preprocess_NE(fileUnlabelled)

started to extract general NE from text....


In [172]:
dsnames = []

'''Extract All the seed names'''
corpuspath = ROOTPATH + '/evaluation_files_prot/X_Seeds_' + str(numberOfSeeds) + '_' + str(iteration) + '.txt'
with open(corpuspath, "r") as file:
    for row in file.readlines():
        dsnames.append(row.strip())
        propernouns.append(row.strip())

        # this step is used for the iterations >1
# for i in range(1, int(numberOfIteration)+1):
#     with open(ROOTHPATH+'/evaluation_files/' + name + '_Iteration' + str(
#             i) + '_POS_' + str(
#         numberOfSeeds) + '_' + str(iteration) + '.txt', 'r') as file:
#         for row in file.readlines():
#             dsnames.append(row.strip())
#             propernouns.append(row.strip())

'''replace the space between the bigram words with _ (for the word2vec embedding)'''
newpropernouns = []
for pp in propernouns:
    temp = pp.split(' ')
    if len(temp) > 1:
        bigram = list(nltk.bigrams(pp.split()))
        for bi in bigram:
            bi = bi[0].lower() + '_' + bi[1].lower()
            # print(bi)
            newpropernouns.append(bi)
    else:
        newpropernouns.append(pp)

dsnames = [x.lower() for x in dsnames]

#dsnames = [s.translate(str.maketrans('', '', string.punctuation)) for s in dsnames]

sentences_split = [s.lower() for s in newpropernouns]

In [169]:
# use the word2vec model
df, labels_array = build_word_vector_matrix(
    ROOTPATH + "/models/modelword2vecbigram.vec", sentences_split)

In [173]:
'''We cluster all terms extracted from the sentences with respect to their embedding vectors using K-means. 
Silhouette analysis is used to find the optimal number k of clusters. Finally, clusters that contain 
at least one of the seed terms are considered to (only) contain entities the same type (e.g Protein). '''

maxcluster=0
if len(df) >= 9:
    for n_clusters in range(2, 10):

        df = StandardScaler().fit_transform(df)
        kmeans_model = KMeans(n_clusters=n_clusters, max_iter=300, n_init=100)
        kmeans_model.fit(df)
        cluster_labels = kmeans_model.labels_

        cluster_to_words = find_word_clusters(labels_array, cluster_labels)
        print(cluster_to_words)
        cluster_labelss = kmeans_model.fit_predict(df)

        finallist = []
        for c in cluster_to_words:
            counter = dict()

            for word in cluster_to_words[c]:
                counter[word] = 0

            for word in cluster_to_words[c]:
                if word in dsnames:

                    for ww in cluster_to_words[c]:
                        finallist.append(ww.replace('_', ' '))

        try:
            silhouette_avg = silhouette_score(df, cluster_labelss)
            if silhouette_avg > maxcluster:
                maxcluster = silhouette_avg
#                 thefile = open(ROOTPATH + "/evaluation_files_prot/" + name + "Pre_Iteration" + numberOfIteration + "_POS_" + str(
#                         numberOfSeeds) + "_" + str(iteration) + ".txt", 'w')
#                 for item in finallist:
#                     thefile.write("%s\n" % item)
                for item in finallist:
                    print(item)
        except:
            continue

{0: ['figure_s1', 'figure_s4', 'nacl', 'arg', 'kcl', 'nodal', 'fgf', 'nematostella', 'pma', 'tca', 'hepes', 'pgm', 'nadph', 'ipa', 'tunel', 'dataset_s2', 'opn', 'asc', 'sdh', 'aox', 'w303', 'u0126', 'malate', 'idh', 'caf', 'formate', 'fumarate', 'pdh', 'icl', 'low_pi', 'rt_pcr', 'fermentative', 'msxb', 'mdh', 'sb431542', 'ja122', 'not1', 'pcs2', 'ogdc', 'aco', 'blattabacterium', 'mdh1', 'mo2', 'iclr', 'fox2', 'acea', 'dlat', 'genetools', 'acek', 'psmad', 'udpg', 'de_robertis', 'aceb', 'orl1', 'pepk', 'torsv', 'cy110', 'bcne', 'fluoresceinated_dextran', 'idp2'], 1: ['pcr', 'atg', 'ppv', 'pyruvate', 'chordin', 'ppi', 'search_tool', 'nadh', 'coa', 'mgso4', 'mtl', 'otx', 'pepck', 'delta2', 'bge', 'sp6', 'elisa_kit', 'psgs', 'texas_red', 'dmrt1', 'malate_dehydrogenase', 'tgfb', 'isocitrate', 'mo1', 'phosphoenolpyruvate', 'g6pdh', 'poxb', 'pyc', 'fum', 'drosophila_sog', 'fks', 'tetramethyl_rhodamine', 'kuroda', 'fdh', 'mmachine', 'fox3']}
pcr
atg
ppv
pyruvate
chordin
ppi
search tool
nadh
coa

In [174]:
dsnames

['n-cym protein',
 'histidinol-phosphate aminotransferase',
 'chordin',
 'annexin a8',
 'protein yipf4',
 'nociceptin',
 'malate dehydrogenase',
 'd-galactonate dehydratase',
 'tgf-beta 1',
 'dipeptidase 1']

# trainingdata_generation.generate_trainingTE(5, 'term_expansion', str(0), str(number))

In [175]:
# training data generation
# for number in range(10):
#     trainingdata_generation.generate_trainingTE(10, 'term_expansion', str(0), str(number))

In [ ]:
# training the NER model which will be saved in the crf_trained_files folder
ner_training.create_austenprop(5, 'term_expansion', str(0))

#
ner_training.train(5, 'term_expansion', str(0))

# ner_training.create_austenprop

In [11]:
import re

outputfile = open(ROOTPATH + '/data/austen.prop', 'r')
text = outputfile.read()
print(text)
name, numberOfIteration, numberOfSeeds, iteration = 'term_expansion', str(33), str(33), str(33)

modifiedpath = ('trainFile=' + ROOTPATH + '/evaluation_files_prot/' + name + '_text_iteration' + numberOfIteration + 
                '_splitted' + str(numberOfSeeds) + '_' + str(iteration) + '.txt')
modifiedpathtest = ('testFile=' + ROOTPATH + '/evaluation_files_prot/' + name + '_text_iteration' + numberOfIteration +
                    'test_splitted' + str(numberOfSeeds) + '_' + str(iteration) + '.txt')
serializeTo = ('serializeTo=' + ROOTPATH + '/crf_trained_files/' + name + '_text_iteration' + numberOfIteration + 
                '_splitted' + str(numberOfSeeds) + '_' + str(iteration) + '.ser.gz')

edited = re.sub(r'trainFile.*?txt', modifiedpath, text, flags=re.DOTALL)
edited = re.sub(r'testFile.*?txt', modifiedpathtest, edited, flags=re.DOTALL)
edited = re.sub(r'serializeTo.*?gz', serializeTo, edited, flags=re.DOTALL)
print(edited)

# location of the training file



trainFile=/Users/sepidehmesbah/PycharmProjects/NERDetector/evaluation_filesMet/embeddingClusteringAll1MET_text_iteration0_splitted5_0.txt
testFile=/Users/sepidehmesbah/PycharmProjects/NERDetector/evaluation_filesMet/embeddingClusteringAll1MET_text_iteration0test_splitted5_0.txt

serializeTo=/Users/sepidehmesbah/PycharmProjects/NERDetector/crf_trained_filesMet/embeddingClusteringAll1MET_text_iteration0_splitted5_0.ser.gz




# structure of your training file; this tells the classifier that
# the word is in column 0 and the correct answer is in column 1
#map = word=0,pos=1,answer=3
map = word=0,answer=1

# This specifies the order of the CRF: order 1 means that features
# apply at most to a class pair of previous class and current class
# or current class and next class.
maxLeft=1

# these are the features we'd like to train with
# some are discussed below, the rest can be
# understood by looking at NERFeatureFactory
useClassFeature=true
useWord=true
us